In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold


In [2]:
# --- 1. Синтетические данные с большим числом признаков ---
X, y = make_classification(n_samples=1000, n_features=50, n_informative=10, random_state=42)
df = pd.DataFrame(X, columns=[f"f{i}" for i in range(X.shape[1])])
df['target'] = y
# Добавим "плохие" признаки
df['constant_feature'] = 1  # один уникальный
df['high_cardinality'] = [f'value_{i}' for i in range(len(df))]  # уникальное значение на строку
df['mostly_zero'] = np.random.choice([0, 1], size=len(df), p=[0.98, 0.02])  # низкая дисперсия
df['duplicated'] = df['f0'] * 1.0  # дублирующий признак


df['missing_col'] = df['f0']
df.loc[:600, 'missing_col'] = np.nan  # много пропусков


In [3]:
df.tail()

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,...,f46,f47,f48,f49,target,constant_feature,high_cardinality,mostly_zero,duplicated,missing_col
995,-1.418978,0.003551,-0.790590,-0.426383,-0.288728,-0.798265,1.048459,0.717391,1.205025,-0.063768,...,1.012702,-1.043291,-1.030703,0.355974,1,1,value_995,0,-1.418978,-1.418978
996,-1.938014,-2.555005,0.528159,-0.298689,-0.316667,-0.535982,3.883104,2.339696,1.093509,-0.592256,...,0.907410,0.158239,1.543237,1.278155,1,1,value_996,0,-1.938014,-1.938014
997,1.299276,1.649182,0.164144,-1.015463,-0.828559,-2.978675,-0.366979,-0.615384,0.314863,1.347091,...,0.663711,-1.387003,1.876140,0.500480,0,1,value_997,0,1.299276,1.299276
998,-2.099545,-1.123901,0.283225,-0.621159,1.791782,-0.707764,-1.722973,0.562720,-0.640689,0.674144,...,1.087940,-0.022644,-1.915869,0.169827,0,1,value_998,0,-2.099545,-2.099545
999,-3.882800,2.177293,-1.710175,-0.399257,2.580292,-2.451362,-1.780405,0.133942,-1.201706,0.632583,...,-0.497370,-0.611189,0.906871,0.841041,0,1,value_999,0,-3.882800,-3.882800


In [4]:
# --- 2. Деление на train и test ---
train, test = train_test_split(df, test_size=0.3, random_state=42, stratify=df['target'])

# Отделим признаки от целевой переменной
X_train = train.drop(columns=['target'])
y_train = train['target']
X_test = test.drop(columns=['target'])
y_test = test['target']

In [5]:
# --- 3. Отбор признаков ---

# 3.1 Удалим признаки с одним уникальным значением
n_unique = X_train.nunique()
to_drop_unique = n_unique[n_unique == 1].index.tolist()
to_drop_unique



['constant_feature']

In [6]:
# 3.2 Удалим признаки с высокой кардинальностью (например, > 90% уникальных)
cardinality = X_train.select_dtypes(exclude='number').nunique() / len(X_train)
to_drop_cardinality = cardinality[cardinality > 0.9].index.tolist()
to_drop_cardinality

['high_cardinality']

In [7]:
# 3.3 Удалим признаки с низкой дисперсией (по умолчанию threshold=0)
vt = VarianceThreshold(threshold=0.1)  # можно варьировать
vt.fit(X_train.select_dtypes(include='number').fillna(0))  # fillna, чтобы избежать ошибок
low_variance_mask = vt.get_support()
to_keep_variance = X_train.select_dtypes(include='number').columns[low_variance_mask].tolist()
to_drop_variance = list(set(X_train.select_dtypes(include='number').columns) - set(to_keep_variance))
to_drop_variance

['constant_feature', 'mostly_zero']

In [8]:
# 3.4 Удалим сильно коррелированные признаки (|corr| > 0.95)
corr_matrix = X_train.select_dtypes(include='number').corr().abs()
upper_triangle = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
to_drop_corr = [column for column in upper_triangle.columns if any(upper_triangle[column] > 0.95)]
to_drop_corr

['duplicated', 'missing_col']

In [9]:
# 3.5 Удалим признаки с большим количеством пропусков (>30%)
missing_frac = X_train.isnull().mean()
to_drop_missing = missing_frac[missing_frac > 0.3].index.tolist()
to_drop_missing

['missing_col']

In [10]:
# Объединяем признаки для удаления ---
to_drop_total = set(
    to_drop_unique +
    to_drop_cardinality +
    to_drop_variance +
    to_drop_corr +
    to_drop_missing
)

print(f"Будет удалено {len(to_drop_total)} признаков:\n{to_drop_total}")

Будет удалено 5 признаков:
{'constant_feature', 'duplicated', 'missing_col', 'high_cardinality', 'mostly_zero'}


In [11]:
# Применим фильтрацию к train и test ---
X_train_filtered = X_train.drop(columns=to_drop_total)
X_test_filtered = X_test.drop(columns=to_drop_total)
print(f"Размерность после фильтрации: {X_train_filtered.shape}")

Размерность после фильтрации: (700, 50)


In [12]:
y_train

,target
736,1
929,0
213,1
618,0
802,1
...,...
656,0
160,1
948,0
40,0


discrete_features=False — указываем, что все признаки считаются не категориальными, а непрерывными. Это важно: если бы были категориальные, надо было бы указать True или список масок.

**Теория: Взаимная информация (Mutual Information)**

Взаимная информация — это мера того, насколько знание одной переменной уменьшает неопределённость другой.

**Формула взаимной информации для дискретных переменных**

$$
I(X; Y) = \sum_{x \in X} \sum_{y \in Y} p(x, y) \cdot \log \left( \frac{p(x, y)}{p(x)p(y)} \right)
$$

где:
- $ X $ — признак,
- $ Y $ — целевая переменная (таргет),
- $ p(x, y) $ — совместное распределение вероятностей $X$ и $Y$,
- $ p(x) $, $ p(y) $ —  распределения вероятностей $X$ и $Y$ соответственно.

---

**Интерпретация**

- Если $ X $ и $ Y $ независимы, то

$$
p(x, y) = p(x) p(y) \implies I(X; Y) = 0
$$

- Если $ X $ полностью определяет $ Y $, то $ I(X; Y) $ достигает максимума.

---

Почему можно удалять признаки с низкой взаимной информацией?

Признаки с низкой взаимной информацией с таргетом:

- Почти независимы от целевой переменной,
- Не дают полезной информации для предсказания,
- Могут увеличивать шум, усложнять модель и способствовать переобучению.

---

Эвристика порога

Обычно признаки с $ I(X; Y) < 0.01 $ считаются малоинформативными и могут быть удалены.


In [13]:
from sklearn.feature_selection import mutual_info_classif

# Вычислим взаимную информацию между признаками и бинарным target
mi = mutual_info_classif(X_train_filtered.fillna(0), y_train, discrete_features=False, random_state=42)

mi_series = pd.Series(mi, index=X_train_filtered.columns)
# Оставим только признаки с MI >= 0.01
to_drop_mi = mi_series[mi_series < 0.01].index.tolist()
to_drop_mi

['f1',
 'f2',
 'f3',
 'f4',
 'f6',
 'f7',
 'f9',
 'f10',
 'f11',
 'f12',
 'f13',
 'f14',
 'f15',
 'f16',
 'f17',
 'f18',
 'f22',
 'f23',
 'f24',
 'f25',
 'f27',
 'f28',
 'f29',
 'f30',
 'f32',
 'f33',
 'f38',
 'f40',
 'f43',
 'f44',
 'f45',
 'f46',
 'f47',
 'f49']

In [17]:
X_train_filtered.columns

Index(['f0', 'f5', 'f8', 'f19', 'f20', 'f21', 'f26', 'f31', 'f34', 'f35',
       'f36', 'f37', 'f39', 'f41', 'f42', 'f48'],
      dtype='object')

In [18]:

print(f"Размерность после фильтрации: {X_train_filtered.shape}")

Размерность после фильтрации: (700, 16)


In [15]:
X_train_filtered

,f0,f5,f8,f19,f20,f21,f26,f31,f34,f35,f36,f37,f39,f41,f42,f48
736,1.121448,3.176310,1.041021,-0.412515,-0.002981,-3.333657,-0.039361,0.264949,-1.103864,-0.143156,-0.217106,-0.440753,1.761432,-1.210699,-3.156575,0.918231
929,0.299562,1.613983,1.220109,-0.766453,0.358879,-0.386291,-0.961883,-2.397585,0.300600,3.373692,0.440567,-5.062486,1.868833,-0.776092,-9.049934,0.496574
213,3.672222,-5.042554,-0.148912,-1.191115,-2.268739,-2.272919,-0.980452,0.262578,0.010386,1.324722,-0.549708,-0.177619,1.323018,-6.456861,-1.851177,-1.132277
618,1.354326,-0.578764,-1.930093,1.542888,-1.742054,3.127651,0.527439,0.649581,-0.815592,1.946678,0.331964,-4.512545,-0.261866,5.808048,0.210095,0.854369
802,0.421160,-0.847923,-0.811125,1.655099,1.982906,-1.892927,-0.044646,-0.028535,0.495075,3.080551,-1.415964,0.623936,-3.946452,-7.899757,-3.209183,-1.682296
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
656,1.106813,-0.632179,0.372464,0.718332,-0.412161,-0.609881,0.340865,-0.922977,-0.766870,1.318534,-0.294094,0.403027,-0.786007,-0.244950,2.223377,0.456203
160,-0.481762,0.138355,-1.349856,-1.617664,1.400647,-2.547325,0.238377,-0.746521,0.990803,-0.697165,0.071778,-0.594270,2.863737,4.352231,1.233812,-0.739629
948,-3.081635,-1.908530,-1.680506,0.362184,0.683063,3.537669,1.569686,0.959576,-1.143671,3.555589,1.703271,1.379354,-1.115249,0.767439,-2.565735,-0.293693
40,-1.658968,-4.179003,-0.753294,-0.428428,1.360295,3.529264,-0.468478,-1.724985,0.140426,5.860456,-1.244436,1.945678,0.614167,-1.613867,-5.273519,-0.428152
